<a href="https://colab.research.google.com/github/Mihail-Chr/projects/blob/main/ML/car_cost/car_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install catboost
! pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor, PassiveAggressiveRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np

RANDOM_STATE = 42

def cv_model(clf, X, y):
    # Задаём сетки гиперпараметров только для тех моделей, которые были в коде
    param_grid = {}
    clf_name = clf.__class__.__name__

    if clf_name == 'SVR':
        param_grid = {'C': np.arange(0.1, 1.2, 0.4)}
    elif clf_name == 'Ridge':
        param_grid = {'alpha': np.arange(0.3, 1, 0.1)}

    if param_grid:
        grid = GridSearchCV(
            clf,
            param_grid=param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            n_jobs=-1,
            verbose=0
        )
        grid.fit(X, y)
        best_clf = grid.best_estimator_
        best_params = grid.best_params_
        best_score = -grid.best_score_
        fit_time = np.mean(grid.cv_results_['mean_fit_time'])
        score_time = np.mean(grid.cv_results_['mean_score_time'])
    else:
        # Для моделей без гиперпараметров просто кросс-валидация со средним RMSE
        scores = cross_val_score(clf, X, y, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
        best_clf = clf.fit(X, y)  # тренируем на всех данных
        best_params = {}
        best_score = -np.mean(scores)
        fit_time = None
        score_time = None

    print(f"\n{clf_name}")
    print(f"Best params: {best_params}")
    print(f"Best RMSE: {best_score:.4f}")
    if fit_time is not None:
        print(f"Avg fit time: {fit_time:.3f} sec, avg predict time: {score_time:.3f} sec")

    return {
        'model': clf_name,
        'best_params': best_params,
        'rmse': best_score,
        'fit_time': fit_time,
        'score_time': score_time
    }

def cross_validate_all_models(X, y):
    results = []
    models = [
        DummyRegressor(),
        Ridge(max_iter=300, random_state=RANDOM_STATE),
        KNeighborsRegressor(n_neighbors=80, n_jobs=-1),
        LinearRegression(),
        RandomForestRegressor(n_estimators=50, warm_start=True, n_jobs=-1, random_state=RANDOM_STATE),
        SVR(max_iter=200),
        SGDRegressor(alpha=1e-4, n_iter_no_change=3, early_stopping=True),
        PassiveAggressiveRegressor(),
        CatBoostRegressor(iterations=1000, learning_rate=0.001, depth=6,
                          loss_function='RMSE', random_seed=RANDOM_STATE, early_stopping_rounds=400,
                          allow_writing_files=False, verbose=False),
        LGBMRegressor(metric='neg_root_mean_squared_error', random_state=RANDOM_STATE, verbose=0)
    ]

    for clf in models:
        print("="*80)
        res = cv_model(clf, X, y)
        results.append(res)
    return results

# --- Пример использования ---
# X, y — подготовленные признаки и целевая переменная (например, из подготовленного polars -> pandas -> sklearn pipeline)
# results = cross_validate_all_models(X, y)


In [ ]:
import time
import polars as pl
import cupy as cp
import numba
from numba import njit, prange
from cuml.ensemble import RandomForestRegressor
from cuml.preprocessing import LabelEncoder
from cuml.model_selection import train_test_split
from cuml.metrics import mean_squared_error

# Загрузка и предобработка данных
def load_and_preprocess_data():
    # Загрузка данных с помощью Polars
    df = pl.read_csv('autos.csv', sep=',', decimal='.')

    # Предобработка данных
    @njit(parallel=True)
    def preprocess(df):
        # Обработка пропусков
        df = df.fill_null(strategy="mean")

        # Кодирование категориальных признаков
        cat_cols = df.select_dtypes(include=['category']).columns
        df = df.with_columns(
            pl.col(col).cast(pl.UInt32) for col in cat_cols
        )

        # Нормализация числовых признаков
        num_cols = df.select_dtypes(include=['float64', 'int64']).columns
        df = df.with_columns(
            (pl.col(col) - pl.col(col).mean()) / pl.col(col).std()
            for col in num_cols
        )

        return df

    df = preprocess(df)
    return df

# Основная функция
def main():
    start_time = time.time()

    # Загрузка и обработка данных
    df = load_and_preprocess_data()

    # Разделение на признаки и целевую переменную
    X = df.drop_columns('Price')
    y = df['Price']

    # Преобразование в Cupy
    X_cupy = cp.array(X.to_numpy())
    y_cupy = cp.array(y.to_numpy())

    # Разделение на train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_cupy, y_cupy, test_size=0.2, random_state=42
    )

    # Инициализация модели
    model = RandomForestRegressor(n_estimators=100, max_depth=10)

    # Обучение модели
    train_start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - train_start

    # Предсказание
    predict_start = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - predict_start

    # Оценка модели
    mse = mean_squared_error(y_test, y_pred)

    # Вывод результатов
    total_time = time.time() - start_time
    print(f"Общее время выполнения: {total_time:.2f} секунд")
    print(f"Время обучения: {train_time:.2f} секунд")
    print(f"Время предсказания: {predict_time:.2f} секунд")
    print(f"MSE: {mse:.2f}")

if __name__ == "__main__":
    main()


In [ ]:
 import time
import polars as pl
import cudf
import cupy as cp
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from cuml.linear_model import LinearRegression as cuLinearRegression, Ridge as cuRidge
from cuml.neighbors import KNeighborsRegressor as cuKNeighborsRegressor
from cuml.ensemble import RandomForestRegressor as cuRandomForestRegressor
from cuml.preprocessing import StandardScaler as cuStandardScaler
from catboost import CatBoostRegressor
import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore')
RANDOM_STATE = 255

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
@njit
def transform_bool(data, col, true_val, false_val):
    return (data == true_val).astype(np.int32)

In [ ]:
def preprocess_data(df):
    # Обработка пропусков
    df = df.with_columns([
        pl.col("VehicleType").fill_null("unknown"),
        pl.col("Gearbox").fill_null("unknown"),
        pl.col("Model").fill_null("unknown"),
        pl.col("FuelType").fill_null("unknown"),
        pl.col("Repaired").fill_null("unknown")
    ])

    # Фильтрация по году регистрации
    df = df.filter(
        (pl.col("RegistrationYear") >= 1950) &
        (pl.col("RegistrationYear") <= 2023)
    )

    # Фильтрация по цене
    df = df.filter(
        (pl.col("Price") >= 100) &
        (pl.col("Price") <= 150000)
    )

    # Преобразование bool
    df = df.with_columns([
        transform_bool(pl.col("Repaired"), "yes", "no").alias("Repaired")
    ])

    # Удаление ненужных столбцов
    df = df.drop(["DateCrawled", "DateCreated", "LastSeen", "NumberOfPictures", "PostalCode"])

    return df

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/car_data.csv')

In [ ]:
df = preprocess_data(df)

cat_col = ['Brand','Model','FuelType','VehicleType','Gearbox','RegistrationMonth','Repaired']
num_col = ['Price','RegistrationYear','Power','Kilometer']

col_auto_df = ['Price','Brand','Model','RegistrationYear','RegistrationMonth',
           'FuelType','VehicleType','Gearbox','Power','Kilometer','Repaired']

interval_cols = num_col
avto_df = df.copy()
avto_df[cat_col] = avto_df[cat_col].astype('category')
# 'PHIK матрица'
display ('PHIK матрица ')

phik_overview = avto_df.phik_matrix( interval_cols=interval_cols)#,n_jobs=-1)
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.3,
    figsize=(10,9))

for i in cat_col:
    if avto_df[i].nunique()<20:
        sns.countplot(y=i, data=avto_df)
        plt.show()

for n in num_col:
    display(n)
    fig, (ax_box, ax_hist) = plt.subplots(2,sharex = True,gridspec_kw = {'height_ratios': (.20, .80)})
    sns.boxplot(x = avto_df[n], ax = ax_box)
    plt.hist(avto_df[n],bins=100)
    ax_box.set(xlabel = '')
    ax_hist.set(xlabel = n)
    ax_hist.set(ylabel = 'count')
    plt.show()

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df.drop('Price')
y = df['Price']
# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Подготовка данных для моделей на CPU и GPU
# Для CPU моделей: оставляем в pandas (или numpy)
# Для GPU моделей: конвертируем в cuDF
# Конвертируем X_train, X_test, y_train, y_test в cuDF
X_train_cudf = cudf.DataFrame.from_pandas(X_train)
X_test_cudf = cudf.DataFrame.from_pandas(X_test)
y_train_cudf = cudf.Series(y_train.values)  # предполагая, что y_train - pandas Series
y_test_cudf = cudf.Series(y_test.values)

In [ ]:
models_cpu = {
     "Dummy": DummyRegressor(),
     "SVR": SVR(),
     "LinearRegression": LinearRegression(),
     "Ridge": Ridge(),
     "PassiveAggressive": PassiveAggressiveRegressor(),
     "SGD": SGDRegressor(),
     "KNeighbors": KNeighborsRegressor(),
     "RandomForest": RandomForestRegressor(),
 }
# Модели, которые будем запускать на GPU через cuML
models_gpu_ml = {
     "cuLinearRegression": cuLinearRegression(),
     "cuRidge": cuRidge(),
     "cuKNeighbors": cuKNeighborsRegressor(),
     "cuRandomForest": cuRandomForestRegressor(),
 }
# Модели, которые могут использовать GPU через свои реализации (CatBoost, LGBM)
models_gpu_other = {
     "CatBoost": CatBoostRegressor(verbose=0, allow_writing_files=False, task_type='GPU'),
     "LGBM": LGBMRegressor(device='gpu'),
 }

In [ ]:
 results = []
  # 1. Обучение моделей на CPU (используем X_train, y_train в формате pandas)
 for name, model in models_cpu.items():
        start = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start
        start = time.time()
        y_pred = model.predict(X_test)
        predict_time = time.time() - start
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        results.append({
            'model': name,
            'train_time': train_time,
            'predict_time': predict_time,
            'rmse': rmse,
            'mae': mae,
            'type': 'CPU'
        })

# 2. Обучение моделей на GPU через cuML (используем X_train_cudf, y_train_cudf)
for name, model in models_gpu_ml.items():
     start = time.time()
     model.fit(X_train_cudf, y_train_cudf)
     train_time = time.time() - start
     start = time.time()
     y_pred = model.predict(X_test_cudf)
     predict_time = time.time() - start
     # Переведем предсказания в numpy для метрик
     y_pred_np = y_pred.to_array()  # для cuML
     rmse = np.sqrt(mean_squared_error(y_test, y_pred_np))
     mae = mean_absolute_error(y_test, y_pred_np)
     results.append({
         'model': name,
         'train_time': train_time,
         'predict_time': predict_time,
         'rmse': rmse,
         'mae': mae,
         'type': 'GPU (cuML)'
     })
# 3. Обучение CatBoost и LGBM на GPU (но данные в pandas)
for name, model in models_gpu_other.items():
     start = time.time()
     model.fit(X_train, y_train)
     train_time = time.time() - start
     start = time.time()
     y_pred = model.predict(X_test)
     predict_time = time.time() - start
     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
     mae = mean_absolute_error(y_test, y_pred)
     results.append({
         'model': name,
         'train_time': train_time,
         'predict_time': predict_time,
         'rmse': rmse,
         'mae': mae,
         'type': 'GPU (other)'
     })
# Соберем результаты в DataFrame
results_df = pl.DataFrame(results)